In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import numpy as np
from sklearn.metrics import make_scorer,mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor,BaggingRegressor

In [30]:
df = pd.read_csv('Crop_Yield_Data_challenge_2.csv')
df.head()

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,15-07-2022,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,15-07-2022,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,15-07-2022,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,14-07-2022,3.30,6400


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 8 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   District                                        557 non-null    object 
 1   Latitude                                        557 non-null    float64
 2   Longitude                                       557 non-null    float64
 3   Season(SA = Summer Autumn, WS = Winter Spring)  557 non-null    object 
 4   Rice Crop Intensity(D=Double, T=Triple)         557 non-null    object 
 5   Date of Harvest                                 557 non-null    object 
 6   Field size (ha)                                 557 non-null    float64
 7   Rice Yield (kg/ha)                              557 non-null    int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 34.9+ KB


In [32]:
#finding out number of unique values in column District

df['District'].nunique()

3

In [33]:
# Creating dummy variables for District column
dummies = pd.get_dummies(df['District'])
df = pd.concat([df, dummies], axis=1)
df.drop(['District'],axis = 1,inplace = True)
df.head()

,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha),Chau_Phu,Chau_Thanh,Thoai_Son
0,10.510542,105.248554,SA,T,15-07-2022,3.40,5500,1,0,0
1,10.509150,105.265098,SA,T,15-07-2022,2.43,6000,1,0,0
2,10.467721,105.192464,SA,D,15-07-2022,1.95,6400,1,0,0
3,10.494453,105.241281,SA,T,15-07-2022,4.30,6000,1,0,0
4,10.535058,105.252744,SA,D,14-07-2022,3.30,6400,1,0,0


In [34]:
#dropping the seasons columns as is in the data description
df.drop(['Season(SA = Summer Autumn, WS = Winter Spring)'],axis = 1,inplace = True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 9 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Latitude                                 557 non-null    float64
 1   Longitude                                557 non-null    float64
 2   Rice Crop Intensity(D=Double, T=Triple)  557 non-null    object 
 3   Date of Harvest                          557 non-null    object 
 4   Field size (ha)                          557 non-null    float64
 5   Rice Yield (kg/ha)                       557 non-null    int64  
 6   Chau_Phu                                 557 non-null    uint8  
 7   Chau_Thanh                               557 non-null    uint8  
 8   Thoai_Son                                557 non-null    uint8  
dtypes: float64(3), int64(1), object(2), uint8(3)
memory usage: 27.9+ KB


In [36]:
# Creating dummy variables forthe column
dummies = pd.get_dummies(df['Rice Crop Intensity(D=Double, T=Triple)'])
df = pd.concat([df, dummies], axis=1)
df.drop(['Rice Crop Intensity(D=Double, T=Triple)'],axis = 1,inplace = True)
df.head()

,Latitude,Longitude,Date of Harvest,Field size (ha),Rice Yield (kg/ha),Chau_Phu,Chau_Thanh,Thoai_Son,D,T
0,10.510542,105.248554,15-07-2022,3.40,5500,1,0,0,0,1
1,10.509150,105.265098,15-07-2022,2.43,6000,1,0,0,0,1
2,10.467721,105.192464,15-07-2022,1.95,6400,1,0,0,1,0
3,10.494453,105.241281,15-07-2022,4.30,6000,1,0,0,0,1
4,10.535058,105.252744,14-07-2022,3.30,6400,1,0,0,1,0


In [37]:

# Convert column named 'date_column' from object to datetime format
df['Date of Harvest'] = pd.to_datetime(df['Date of Harvest'], format='%d-%m-%Y')

#seperating the column into 3 columns harvest day, month,year
df['Harvest day'] = df['Date of Harvest'].dt.day
df['Harvest month']  = df['Date of Harvest'].dt.month
df['Harvest year'] = df['Date of Harvest'].dt.year
df.drop(columns=['Date of Harvest'], inplace=True)
df.head()


,Latitude,Longitude,Field size (ha),Rice Yield (kg/ha),Chau_Phu,Chau_Thanh,Thoai_Son,D,T,Harvest day,Harvest month,Harvest year
0,10.510542,105.248554,3.40,5500,1,0,0,0,1,15,7,2022
1,10.509150,105.265098,2.43,6000,1,0,0,0,1,15,7,2022
2,10.467721,105.192464,1.95,6400,1,0,0,1,0,15,7,2022
3,10.494453,105.241281,4.30,6000,1,0,0,0,1,15,7,2022
4,10.535058,105.252744,3.30,6400,1,0,0,1,0,14,7,2022


In [38]:

# Function to compute the adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    
    r2 = r2_score(targets, predictions)
    
    n = predictors.shape[0]
    
    k = predictors.shape[1]
    
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# Function to compute MAPE
def mape_score(targets, predictions):
    
    return np.mean(np.abs(targets - predictions) / targets) * 100


# MAPE
def mape(predictions, targets):
    return np.mean(np.abs((targets - predictions)) / targets) * 100


# MAE
def mae(predictions, targets):
    return np.mean(np.abs((targets - predictions)))

  
# RMSE
def rmse(predictions, targets):
    return np.sqrt(((targets - predictions) ** 2).mean())


# Function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):

    # Predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)                      # To compute R-squared
    
    adjr2 = adj_r2_score(predictors, target, pred)   # To compute adjusted R-squared
    
    rmse = np.sqrt(mean_squared_error(target, pred)) # To compute RMSE
    
    mae = mean_absolute_error(target, pred)          # To compute MAE
    
    mape = mape_score(target, pred)                  # To compute MAPE

    # Creating a dataframe of metrics
    
    df_perf = pd.DataFrame(
        {
            "RMSE":  rmse,
            "MAE":  mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        
        index = [0],
    )

    return df_perf


def model_pref(olsmodel, x_train, x_test, y_train,y_test):

    # Prediction on the training data
    y_pred_train = olsmodel.predict(x_train)
    y_observed_train = y_train

    # Prediction on the test data
    y_pred_test = olsmodel.predict(x_test)
    y_observed_test = y_test

    print(
        pd.DataFrame(
            {
                "Data": ["Train", "Test"],
                "RMSE": [
                    rmse(y_pred_train, y_observed_train),
                    rmse(y_pred_test, y_observed_test),
                ],
                
                "MAE": [
                    mae(y_pred_train, y_observed_train),
                    mae(y_pred_test, y_observed_test),
                ],
                
                "MAPE": [
                    mape(y_pred_train, y_observed_train),
                    mape(y_pred_test, y_observed_test),
                ],
            }
        )
    ) 

In [56]:
#seperating the feature variables and target variables
x = df.drop('Rice Yield (kg/ha)',axis = 1)

y = df['Rice Yield (kg/ha)']


#splitting the training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True, random_state = 1)

#checking the shape of the train and test set
print('Train Shape:',x_train.shape)
print('Test Shape:',x_test.shape)

Train Shape: (445, 11)
Test Shape: (112, 11)


In [40]:
#fitting the OLS model 
import statsmodels.api as sm

# Statsmodel api does not add a constant by default, we need to add it explicitly
x_train1 = sm.add_constant(x_train)

# Add constant to the test data
x_test1 = sm.add_constant(x_test)

# Create the model
olsmodel1 = sm.OLS(y_train, x_train1).fit()

# Get the model summary
olsmodel1.summary()

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Rice Yield (kg/ha)   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     107.5
Date:                Thu, 02 Mar 2023   Prob (F-statistic):           3.88e-98
Time:                        11:28:27   Log-Likelihood:                -3368.3
No. Observations:                 445   AIC:                             6755.
Df Residuals:                     436   BIC:                             6791.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Latitude          961.3608    552.551      1.740      0.083    -124.634    2047.356
Longitude        -104.8885    347.555     -0.302      0.763    -787.979     578.202
Field size (ha)   -19.5135     18.659     -1.046      0.296     -56.186      17.159
Chau_Phu         -118.4484     85.038     -1.393      0.164    -285.585      48.688
Chau_Thanh          2.0086     37.938      0.053      0.958     -72.555      76.572
Thoai_Son         116.4422     77.170      1.509      0.132     -35.230     268.114
D                 -43.6900     31.857     -1.371      0.171    -106.303      18.923
T                  43.6924     31.857      1.372      0.171     -18.919     106.304
Harvest day       -15.5886      3.798     -4.104      0.000     -23.054      -8.123
Harvest month    -364.5791     12.521    -29.118      0.000    -389.187    -339.971
Harvest year        4.8789     18.455      0.264      0.792     -31.394      41.152
==============================================================================
Omnibus:                       49.103   Durbin-Watson:                   2.174
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.562
Skew:                           0.227   Prob(JB):                     0.000154
Kurtosis:                       2.139   Cond. No.                     2.69e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.52e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [41]:
lin_reg_test = model_performance_regression(olsmodel1, x_test1, y_test)
lin_reg_test

,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,440.094339,370.979696,0.63417,0.593929,5.759566


In [42]:
#comparing the performance on train and test set to ensure theres not overfitting
model_pref(olsmodel1, x_train1, x_test1, y_train,y_test)

    Data        RMSE         MAE      MAPE
0  Train  468.838595  390.247267  5.870752
1   Test  440.094339  370.979696  5.759566


In [57]:
#RandomForest regressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 1)

# Fitting the model
regressor.fit(x_train, y_train)

# Model Performance on the test data
regressor_perf_test = model_performance_regression(regressor, x_test, y_test)

regressor_perf_test

,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,469.241823,377.785714,0.584107,0.538359,5.836971


In [58]:
#comparing the performance on train and test set to ensure theres not overfitting
model_pref(regressor, x_train, x_test, y_train,y_test)

    Data        RMSE         MAE      MAPE
0  Train  188.156006  150.507416  2.244368
1   Test  469.241823  377.785714  5.836971


there is overfitting occuring

In [72]:
# Retrieve selected columns from the original dataframe that have coef of >0.05
alt_df = df[['Latitude', 'Chau_Thanh', 'Thoai_Son', 'T', 'Harvest year', 'Rice Yield (kg/ha)']]


X = alt_df.drop('Rice Yield (kg/ha)',axis = 1)

Y = alt_df['Rice Yield (kg/ha)']

#splitting the training and test sets
x_train2, x_test2, y_train2, y_test2 = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

#checking the shape of the train and test set
print('Train Shape:',x_train2.shape)
print('Test Shape:',x_test2.shape)

Train Shape: (445, 5)
Test Shape: (112, 5)


In [76]:
#RandomForest regressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 1)

# Fitting the model
regressor.fit(x_train, y_train)

# Model Performance on the test data
regressor_perf_test = model_performance_regression(regressor, x_test, y_test)

regressor_perf_test

,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,469.241823,377.785714,0.584107,0.538359,5.836971


In [77]:
#comparing the performance on train and test set to ensure theres not overfitting
model_pref(regressor, x_train, x_test, y_train,y_test)

    Data        RMSE         MAE      MAPE
0  Train  188.156006  150.507416  2.244368
1   Test  469.241823  377.785714  5.836971


In [88]:
from sklearn.model_selection import GridSearchCV
#hyperparameter tuning using grid search
rf_tuned = RandomForestRegressor(random_state = 1)

# Define the hyperparameters to be tuned
param_grid = {
    "n_estimators": [100, 110, 120, 130, 140],
    "max_depth": [3, 5, 7, 9, 11],
    "max_features": [0.5, 0.7, 0.8, 0.9, 1],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],

}


# Type of scoring used to compare parameter combinations
scorer = make_scorer(r2_score)

# Run the grid search
grid_obj = GridSearchCV(rf_tuned, param_grid, scoring = scorer, cv = 10)

grid_obj = grid_obj.fit(x_train, y_train)

# Set the rf_tuned_regressor to the best combination of parameters
rf_tuned_regressor = grid_obj.best_estimator_

rf_tuned_regressor.fit(x_train, y_train)


rf_tuned_regressor_perf_test = model_performance_regression(rf_tuned_regressor, x_test, y_test)

rf_tuned_regressor_perf_test
     

,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,434.255115,354.502085,0.643813,0.604633,5.503417


In [81]:
#comparing the performance on train and test set to ensure theres not overfitting
model_pref(rf_tuned_regressor, x_train, x_test, y_train,y_test)

    Data        RMSE         MAE      MAPE
0  Train  381.868868  310.321277  4.639022
1   Test  444.926422  360.583482  5.575763


In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

#define the model
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()